## <span style='color:green'> Goals of the project </span>
This is a Data Cleaning and merging operation aimed at putting all Flight Data sets together and finally cutting them into **Depature** and **Arrival** flight schedules. 

It has different Data Bases which will be cleaned and then further merged . At the end of , the files will be stored and further Exploration Data Anaylsis will be deployed . 

In [1]:
import numpy  as np # importing the necessary libararies for the given project 
import pandas as pd 
from matplotlib import pyplot as plt
import seaborn as sns 
import dateutil.parser as parser 
import glob
import time
import csv
from datetime import datetime as dt, timedelta
import warnings
warnings.filterwarnings("ignore")
import re

In [2]:
# Record the start time in order to understand how long the model performs. 
start_time = time.time()

## 1.0 First section of Data cleaning and preparation

### 1.1 Data Importation from saved Directory 
 

In [3]:
# df is saved in a parquet format to 
NF = pd.read_parquet(
    r'D:\DEHLI Machine Learning Projects\AOBD_Historical_Data\NF/merged.parquet'
)


In [4]:
print(f'Shape of the Data frame is {NF.shape}')

Shape of the Data frame is (729971, 28)


### 1.2 The below functions do the following : 
- Renaming and  and deleting out incorrect Terminals
- Converting Dates into the correct Pandas format 
- Creating new columns to represent the Indian local times

In [5]:
def correct_terminals_name(df,column_name): 
    '''
    The goal of this function is to change number terminals to correct terminal names . The data was written in 
    a wring format and for that matter, It has to be changed from nummbers to T-number formats. 
    '''
    if column_name in df.columns:
    # Replace '3' with 'T3' and '2' with 'T2'
        df[column_name] = df[column_name].replace({'3': 'T3', '2': 'T2','C':'T1','D':'T1'})
    return df 

In [6]:
# Applying the correct_terminals_name
NF = correct_terminals_name(NF,"public_terminal") 
# The plan is to create a delete_terminals function to remove the F, x and  non

In [7]:
def delete_incorrect_terminals(df, column_name): 
    '''
    The delete_incorrect_terminals function is aimed at removing values such as F, X, G, 
    as well as None, that do not represent airport terminals.
    '''
    # Convert the column to string and filter out rows where the column's value ends with specified characters
    mask = df[column_name].astype(str).str.endswith((' ', 'X', 'F', 'G', 'J', 'P', 'H'))
    df.drop(df[mask].index, inplace=True)
    
    # Drop rows with NaN values in the specified column
    df.dropna(subset=[column_name], inplace=True)
    return df

In [8]:
NF = delete_incorrect_terminals(NF,"public_terminal") # Applying the function to remove unecessary terminals . 

In [9]:
def convert_date_pandas_format(df, column_name):
    '''
    The goal of this function is to convert the date of flight depature and arrival to a pandas understood format
    '''
    result = pd.to_datetime(df[column_name],
                           # dayfirst=True,
                            format='mixed',
                            errors="coerce")
    return df

In [10]:
NF = convert_date_pandas_format(NF, 'sibt')
NF = convert_date_pandas_format(NF, 'sobt')

In [11]:
import datetime
def timeChange(dt):
    '''
    The goal of this function is to convert the uct time to the ugandan local time. The function returns
    an updated time with an addition time of 5 hours and 30 minutes . 
    '''
    if pd.isna(dt):
        return "NAT"
    else:
        dt = pd.to_datetime(dt,dayfirst=True)  # Convert dt to datetime object
        updated_time = dt + datetime.timedelta(
            hours=5, minutes=30)  #time_change had been taken of by dt..
        return updated_time

In [12]:
NF["sibt_uct"] = NF["sibt"].apply(lambda x: timeChange(x))
NF["sobt_uct"] = NF["sobt"].apply(lambda x: timeChange(x))

### 1.3 The below functions take care of the following tasks 
- Select specific columns that will be used for further Data Manipulation .
- Sub-devide the Data Frame into Arrival and Deapture sets

In [13]:
def choose_specific_legtype(df, leg_type, column_name, C_1, C_2, C_3, C_4, C_5,
                            C_6, C_7, C_8, C_9, C_10):
    '''
    The goal of this function is to sub-devide the df into two parts , Arrival and Deaptures filled with column_name.
    the df represents the Data Frame and leg_type is used to cut the df into those two parts . 
    '''
    filtered_df = df[df[leg_type] == column_name]
    filtered_df[C_1] = filtered_df[C_1].apply(pd.to_datetime)
    filtered_df.loc[:, 'Date'] = filtered_df.loc[:, C_1].dt.date.apply(
        pd.to_datetime)
    filtered_df = filtered_df[[
        C_1, 'Date', C_2, C_3, C_4, C_5, C_6, C_7, C_8, C_9, C_10
    ]]

    return filtered_df

In [14]:
NF_Arrival = choose_specific_legtype(NF, 'leg_type', "ARRIVAL", 'sibt_uct',
                                     'all_capacity_pax', 'flight_number',
                                     'origin_airport_iata',
                                     'destination_airport_iata', 'leg_type',
                                     "traffic_type", "all_boarded_pax",
                                     "public_terminal", "airline_iata")

In [15]:
NF_Depature = choose_specific_legtype(NF, 'leg_type', "DEPARTURE", 'sobt_uct',
                                     'all_capacity_pax', 'flight_number',
                                     'origin_airport_iata',
                                     'destination_airport_iata', 'leg_type',
                                     "traffic_type", "all_boarded_pax",
                                     "public_terminal", "airline_iata")

### 1.4 Importing calender information in order to merge it up with the date

In order to capture holidays for public holidays , I decided to create a function entitled Kalender_Holiday showcasing all the Indian holidays, this is very important because it shades light on how events of this sort affect flight patterns . This is in two forms : 

- 1 : `´Passanger load aka pax load:`´ The represents the percentage of passangers on a Flight 
- 2: `´ Total Flights:`´ This represents the number of flights around a particular holidays or season .

In [16]:
start_date = "2018-01-01"
end_date = "2023-12-31"
date_range = pd.date_range(start=start_date, end=end_date)
# Create a DataFrame with Date column
Kalender = pd.DataFrame({"Date": date_range})

# Define Indian holidays

# Extract day of the week and create a Weekend column
Kalender['Day of week'] = Kalender["Date"].dt.day_name()
Kalender['Weekend (0/1)'] = np.where(
    Kalender["Date"].dt.dayofweek.isin([5, 6]), 1, 0)
Kalender['Overlap with Weekend'] = np.where(
    Kalender["Date"].dt.dayofweek.isin([5, 6]), 1, 0)
Kalender['Extended weekend'] = np.where(
    Kalender["Date"].dt.dayofweek.isin([5, 6]), 1, 0)
Kalender.head(10)

,Date,Day of week,Weekend (0/1),Overlap with Weekend,Extended weekend
0,2018-01-01,Monday,0,0,0
1,2018-01-02,Tuesday,0,0,0
2,2018-01-03,Wednesday,0,0,0
3,2018-01-04,Thursday,0,0,0
4,2018-01-05,Friday,0,0,0
5,2018-01-06,Saturday,1,1,1
6,2018-01-07,Sunday,1,1,1
7,2018-01-08,Monday,0,0,0
8,2018-01-09,Tuesday,0,0,0
9,2018-01-10,Wednesday,0,0,0


In [17]:
def Kalender_Holiday(Kalender_Date):
    indian_holidays = [
        "2018-01-01 00:00:00",  # New Year
        "2018-01-14 00:00:00",  # Makar Sankranti / Pongal
        "2018-01-26 00:00:00",  # Republic Day
        "2018-02-13 00:00:00",  # Mahashivratri
        "2018-03-02 00:00:00",  # Holi
        "2018-03-18 00:00:00",  # Ugadi / Gudi Padwa
        "2018-03-30 00:00:00",  # Good Friday
        "2018-04-14 00:00:00",  # Baisakhi / Ambedkar Jayanti
        "2018-04-30 00:00:00",  # Buddha Purnima
        "2018-06-16 00:00:00",  # Eid al-Fitr
        "2018-08-15 00:00:00",  # Independence Day
        "2018-08-26 00:00:00",  # Raksha Bandhan
        "2018-09-03 00:00:00",  # Janmashtami
        "2018-09-21 00:00:00",  # Muharram
        "2018-10-02 00:00:00",  # Gandhi Jayanti
        "2018-10-18 00:00:00",  # Dussehra
        "2018-11-06 00:00:00",  # Diwali
        "2018-11-23 00:00:00",  # Guru Nanak Jayanti
        "2018-12-25 00:00:00",  # Christmas
        "2019-01-01 00:00:00",  # New Year
        "2019-01-15 00:00:00",  # Makar Sankranti / Pongal
        "2019-01-26 00:00:00",  # Republic Day
        "2019-03-04 00:00:00",  # Mahashivratri
        "2019-03-21 00:00:00",  # Holi
        "2019-04-06 00:00:00",  # Ugadi / Gudi Padwa
        "2019-04-19 00:00:00",  # Good Friday
        "2019-04-14 00:00:00",  # Baisakhi / Ambedkar Jayanti
        "2019-05-18 00:00:00",  # Buddha Purnima
        "2019-06-05 00:00:00",  # Eid al-Fitr
        "2019-08-15 00:00:00",  # Independence Day
        "2019-08-15 00:00:00",  # Raksha Bandhan
        "2019-08-24 00:00:00",  # Janmashtami
        "2019-09-10 00:00:00",  # Muharram
        "2019-10-02 00:00:00",  # Gandhi Jayanti
        "2019-10-08 00:00:00",  # Dussehra
        "2019-10-27 00:00:00",  # Diwali
        "2019-11-12 00:00:00",  # Guru Nanak Jayanti
        "2019-12-25 00:00:00",  # Christmas
        "2022-01-01 00:00:00",  # New Year
        "2022-01-14 00:00:00",  # Makar Sankranti / Pongal
        "2022-01-26 00:00:00",  # Republic Day
        "2022-02-28 00:00:00",  # Mahashivratri
        "2022-03-18 00:00:00",  # Holi
        "2022-04-02 00:00:00",  # Ugadi / Gudi Padwa
        "2022-04-15 00:00:00",  # Good Friday
        "2022-04-14 00:00:00",  # Baisakhi / Ambedkar Jayanti
        "2022-05-15 00:00:00",  # Buddha Purnima
        "2022-04-02 00:00:00",  # Eid al-Fitr
        "2022-08-15 00:00:00",  # Independence Day
        "2022-08-11 00:00:00",  # Raksha Bandhan
        "2022-08-20 00:00:00",  # Janmashtami
        "2022-08-08 00:00:00",  # Muharram
        "2022-10-02 00:00:00",  # Gandhi Jayanti
        "2022-10-05 00:00:00",  # Dussehra
        "2022-10-24 00:00:00",  # Diwali
        "2022-11-08 00:00:00",  # Guru Nanak Jayanti
        "2022-12-25 00:00:00",  # Christmas
        "2023-01-01 00:00:00",  # New Year
        "2023-01-14 00:00:00",  # Makar Sankranti / Pongal
        "2023-01-26 00:00:00",  # Republic Day
        "2023-02-17 00:00:00",  # Mahashivratri
        "2023-03-08 00:00:00",  # Holi
        "2023-03-28 00:00:00",  # Ugadi / Gudi Padwa
        "2023-03-30 00:00:00",  # Ugadi / Gudi Padwa
        "2023-04-07 00:00:00",  # Good Friday
        "2023-04-14 00:00:00",  # Baisakhi / Ambedkar Jayanti
        '2023-05-05 00:00:00',  # Buddha Purnima
        "2023-03-28 00:00:00",  # Eid al-Fitr
        "2023-08-15 00:00:00",  # Independence Day
        "2023-08-30 00:00:00",  # Raksha Bandhan
        "2023-08-09 00:00:00",  # Janmashtami
        "2023-08-27 00:00:00",  # Muharram
        "2023-10-02 00:00:00",  # Gandhi Jayanti
        "2023-09-25 00:00:00",  # Dussehra
        "2023-10-19 00:00:00",  # Diwali
        "2023-11-26 00:00:00",  # Guru Nanak Jayanti
        "2023-12-25 00:00:00",  # Christmas
    ]
    if str(Kalender_Date) in indian_holidays:
        return 1
    else:

        return 0

In [18]:
# Apply the Kalender_Holiday function to the 'Date' column to get the holiday indicator
Kalender['Holiday (0/1)'] = Kalender['Date'].apply(Kalender_Holiday)

# Identify the rows where the 'Holiday' column is 1
holiday_rows = Kalender[Kalender['Holiday (0/1)'] == 1].index

# Update the 'Extended weekend' column based on the condition you specified
for index in holiday_rows:
    if index - 1 in Kalender.index:
        Kalender.at[index - 1, 'Extended weekend'] = 1
    if index + 1 in Kalender.index:
        Kalender.at[index + 1, 'Extended weekend'] = 1

# Drop the 'Holiday' column if you don't need it anymore
#Kalender = Kalender.drop(columns=['Holiday'])


In [19]:
# Creating a new column entitled Festival representing Public holidays.
Kalender['Festival'] = Kalender['Date'].apply(Kalender_Holiday)
# Converting the Date column to a pandas datetime format
Kalender['Date'] = Kalender['Date'].apply(pd.to_datetime)

### 1.5 Adding 6 more columns to a the Data Frame for pattern study 

In order to study how holidays influence passanger flight patterns , I added 6 new rows representing . In this way, we will be able to group flight numbers and load factors w.r.t the created days .

- 1:`´1day_after :`´ This represents the **1st day** after a public holiday .
- 2:`´2day_after :`´ This represents the **2nd day** after a public holiday .
- 3:`´3day_after :`´ This represents the **3rd day** after a public holiday .

- 4:`´1day_before :`´ This represents the **1st day** before a public holiday .
- 5:`´2day_before :`´ This represents the **2nd day** before a public holiday .
- 6:`´3day_before :`´ This represents the **3rd day** before a public holiday .


In [20]:
# Create a new column '1day_after' and initialize it with 0
Kalender['1day_after'] = 0

# Identify rows where 'Holiday' column is 1
holiday_rows = Kalender['Holiday (0/1)'] == 1

# Set '1day_after' to 1 for the day after a holiday
Kalender.loc[holiday_rows.shift(1, fill_value=False), '1day_after'] = 1
Kalender['1day_after'] = Kalender['1day_after'].astype(int)

In [21]:
# Create a new column '2day_after' and initialize it with 0
Kalender['2day_after'] = 0

# Identify rows where 'Holiday' column is 1
holiday_rows = Kalender['Holiday (0/1)'] == 1

# Set '2day_after' to 2 for the day after a holiday
Kalender.loc[holiday_rows.shift(2, fill_value=False), '2day_after'] = 1
Kalender['2day_after'] = Kalender['2day_after'].astype(int)

In [22]:
# Create a new column '3day_after' and initialize it with 0
Kalender['3day_after'] = 0

# Identify rows where 'Holiday' column is 1
holiday_rows = Kalender['Holiday (0/1)'] == 1

# Set '3day_after' to 1 for the day before a holiday
Kalender.loc[holiday_rows.shift(3, fill_value=False), '3day_after'] = 1
Kalender['3day_after'] = Kalender['3day_after'].astype(int)

In [23]:
Kalender['4day_after'] = 0

# Identify rows where 'Holiday' column is 1
holiday_rows = Kalender['Holiday (0/1)'] == 1

# Set '4day_after' to 1 for the day after a holiday
Kalender.loc[holiday_rows.shift(4, fill_value=False), '4day_after'] = 1
Kalender['4day_after'] = Kalender['4day_after'].astype(int)

In [24]:
Kalender['1day_before'] = 0

# Identify rows where 'Holiday' column is 1
holiday_rows = Kalender['Holiday (0/1)'] == 1

# Set '1day_before' to 1 for the day before a holiday
Kalender.loc[holiday_rows.shift(-1, fill_value=False), '1day_before'] = 1
Kalender['1day_before'] = Kalender['1day_before'].astype(int)

In [25]:
Kalender['2day_before'] = 0

# Identify rows where 'Holiday' column is 1
holiday_rows = Kalender['Holiday (0/1)'] == 1

# Set '2day_before' to 1 for the day before a holiday
Kalender.loc[holiday_rows.shift(-2, fill_value=False), '2day_before'] = 1
Kalender['2day_before'] = Kalender['2day_before'].astype(int)

In [26]:
Kalender['3day_before'] = 0

# Identify rows where 'Holiday' column is 1
holiday_rows = Kalender['Holiday (0/1)'] == 1

# Set '3day_before' to 1 for the day before a holiday
Kalender.loc[holiday_rows.shift(-3, fill_value=False), '3day_before'] = 1
Kalender['3day_before'] = Kalender['3day_before'].astype(int)

In [27]:
Kalender['4day_before'] = 0

# Identify rows where 'Holiday' column is 1
holiday_rows = Kalender['Holiday (0/1)'] == 1

# Set '4day_before' to 1 for the day before a holiday
Kalender.loc[holiday_rows.shift(-4, fill_value=False), '4day_before'] = 1
Kalender['4day_before'] = Kalender['4day_before'].astype(int)

In [28]:
# Merging both the Arrival . Depature data frames with the  Kalender Data Frame.
NF_Arrival_Calender = pd.merge(NF_Arrival, Kalender, on="Date", how="inner")
NF_Depature_Calender=pd.merge(NF_Depature,Kalender,on="Date",how="inner")

In [29]:
def convert_column_int_fillna(df, column_name):
    ''' Converts the specified column to integers and fills null values with 0 '''
    try:
        df[column_name] = df[column_name].apply(pd.to_numeric, errors='coerce')
        df[column_name] = df[column_name].fillna(0).astype(int)
    except ValueError:
        # Handle cases where conversion fails (e.g., non-numeric strings)
        # You can customize this part based on your requirements
        pass
    return df

In [30]:
NF_Arrival_Calender = convert_column_int_fillna(NF_Arrival_Calender,
                                                 'all_capacity_pax')
NF_Arrival_Calender = convert_column_int_fillna(NF_Arrival_Calender,
                                                 'all_boarded_pax')

In [31]:
NF_Depature_Calender = convert_column_int_fillna(NF_Depature_Calender,
                                                 'all_capacity_pax')
NF_Depature_Calender = convert_column_int_fillna(NF_Depature_Calender,
                                                 'all_boarded_pax')

 ## 2.0 Second section of Data cleaning and preparation
- I did recieve another dirty dirty fame that  also needed a fresh cleaning before the two data sets could be put together 

- This includes renaming and deleting of unecessary columns , removal of duplicates,column merge ups , Date conversions among others . 


### 2.1 Importing the First DB into jupyter Notebook

In [33]:
UFISAll = pd.read_parquet(
    r'D:\DEHLI Machine Learning Projects\AOBD_Historical_Data\UFISAll.parquet'
)  # Saved

### 2.2 This section contains the following functions:
- Selection of important colimns 
- Conversion of Dates 
- Merging of flight schedules with the airline Data Set

In [34]:
UFISAll.columns = [
    "ADID", 'FLNO', "ALC3", "HOPO", "STOA", "STOD", "ONBL", "OFBL", "REGN",
    "ACT3", "STYP", "FTYP", "FLTI", "STEV", "PAXT", "RWYA", "RWYD", "URNO",
    "RKEY", "ALC2", "ORG3", "DES3", "AIRB", "LAND", "PSTD", "PSTA", "GTA1",
    "GTD1", "BLT1", "CKIT", "CKIF", "PAXF", "FLDA"
]  # namign the columns

In [35]:
def concatenate_two_columns(df, new_column, column_1, column_2):
    df[new_column] = df[column_1] + df[column_2]
    return df

In [36]:
def remove_unwanted_trailings(df, column_name):
    df[column_name] = df[column_name].str.replace('\n', '')
    return df

In [37]:
def delete_unimportant_features(df, feature_one, feature_two, feature_three,
                                feature_four, feature_five, feature_six,
                                feature_seven, feature_eight, feature_nine,
                                feature_ten, feature_eleven, feature_twelve,
                                feature_thirteen, feature_fourteen,
                                feature_fifteen, feature_sixteen):
    '''
    This function takes in a DataFrame and removes specified columns that are not important for deciding
    a client's ability to receive credit. columns such as Names, customer_ID etc., do not play a part. 
    Users can specify additional features to drop out , more features to be removed will be taken care as time goes
    and this is very important for both of us . 
    '''
    df = df.drop(columns=[
        feature_one, feature_two, feature_three, feature_four, feature_five,
        feature_six, feature_seven, feature_eight, feature_nine, feature_ten,
        feature_eleven, feature_twelve, feature_thirteen, feature_fourteen,
        feature_fifteen, feature_sixteen
    ],
                 inplace=True)
    return df

In [38]:
concatenate_two_columns(UFISAll, "ALACT", "ALC2", "ACT3")
remove_unwanted_trailings(UFISAll, "STOA")
delete_unimportant_features(UFISAll, "RWYA","RWYD","URNO","RKEY","AIRB","LAND","PSTD","PSTA","GTA1","GTD1","BLT1","CKIT","CKIF","PAXF","HOPO","ALC3")

### 2.3 This section includes :
- converting the date time columns into a pandas date time format
- Creation of the local Inidian time and date 
- Removing irrelevant Flights form the data frame.

####  Filter out empty strings and convert to datetime

In [39]:
def remove_empty_convert_datetime(df, column_name):
    '''
    The goal of this function is to remove empty strings and spaces from the charachters, 
    it also converts these specific columns into a datetime pandas object 
    '''
    df[column_name] = df[column_name].astype(str).str.replace('\n',
                                                              '').str.strip()
    df[column_name] = df[column_name][df[column_name] != ''].apply(
        lambda x: pd.to_datetime(x, format='%Y%m%d%H%M%S', errors='coerce'))
    return df

In [40]:
def convert_int_date(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name], format='%Y%m%d')
    return df

In [41]:
UFISAll = remove_empty_convert_datetime(UFISAll, "STOA")
UFISAll = remove_empty_convert_datetime(UFISAll, "STOD")
UFISAll = remove_empty_convert_datetime(UFISAll, "ONBL")
UFISAll = remove_empty_convert_datetime(UFISAll, "OFBL")

In [42]:
UFISAll = convert_int_date(UFISAll,"FLDA")

In [43]:
UFISAll=UFISAll.rename(columns={"FLDA":"Date"})

In [44]:
UFISAll["STOA_UCT"] = UFISAll["STOA"].apply(lambda x: timeChange(x)) 
UFISAll["STOD_UCT"] = UFISAll["STOD"].apply(lambda x: timeChange(x)) 
UFISAll["ONBL_UCT"] = UFISAll["ONBL"].apply(lambda x: timeChange(x))  
UFISAll["OFBL_UCT"] = UFISAll["OFBL"].apply(lambda x: timeChange(x)) 

In [45]:
UFISAll=UFISAll.drop(columns=['STOA','STOD','ONBL','OFBL'])

In [46]:
def select_releveant_legtype(df, column_name, Arrival, Depature):
    '''
    The goal of this function is to select only relevant leg types which are Depature and Arrival flights . 
    '''
    df = df[(df[column_name] == Arrival) | (df[column_name] == Depature)]
    return df

In [47]:
UFISAll = select_releveant_legtype(UFISAll,'ADID','A','D')

In [48]:
UFISAll = correct_terminals_name(UFISAll,"STEV") 

In [49]:
UFISAll = delete_incorrect_terminals(UFISAll,"STEV") #it should return a df

### 2.4 : This section includes :
- Removing irrelevant Service types 
- Removing cancelled flights from the data frame 
- Removing duplicate values 
- Merging the Flight and air craft type Data Frames 

In [50]:
def important_service_types(df, column_name):
    result_df = df[(df[column_name] == 'C ') | (df[column_name] == 'G ') |
                   (df[column_name] == 'J ') | (df[column_name] == 'L ')]
    return result_df

In [51]:
UFISAll = important_service_types(UFISAll,'STYP')

In [52]:
def delete_cancelled_flights(df, column_name):
    '''
    The goal of this function to delete out all flights under the type O from the Flight Schedule Data Frame,
    Flights with O type are represented as cancelled flights .
    '''
    result_df = df[(df[column_name] != 'O')]
    return result_df

In [53]:
def delete_spaces(df, column_name):
    ''' The goal of this function is to remove leading and trailing spaces,
    remove rows where the column contains only spaces or is empty,
    and remove rows with NaN values in the column. '''

    # Remove leading and trailing spaces from the specified column
    df[column_name] = df[column_name].str.strip()

    # Filter out rows where the column is either empty or contains only spaces
    df = df[(df[column_name] != '') | (df[column_name] != ' ')]

    return df

In [54]:
UFISAll = delete_cancelled_flights(UFISAll, 'FTYP')

In [55]:
UFISAll = delete_spaces(UFISAll,'ALC2')

In [56]:
UFISAll= UFISAll.drop_duplicates()

####  <span style='color:blue'>1.4 (Reading Air craft + Airline Records from the DB1) </span>

In [57]:
#importing Calender file for mapping purposes
Aircraft_Sit = pd.read_csv(
    r"D:\DEHLI Machine Learning Projects\AOBD_Historical_Data\AirlineSitCapacity.csv",
    delimiter=';')

In [58]:
Aircraft_Sit['ALACT'] = Aircraft_Sit['Airline'] + Aircraft_Sit['AcftType']
Aircraft_INFO = Aircraft_Sit.rename(
    columns={"AcftType": "ACT3"})  # is to be paired on the second section .

####  <span style='color:blue'>1.41 (Reading Air craft + Airline Records from the DB2) </span>

In [59]:
# Importing Another Airlines - AirlineSitCapacity data set
Aircraft_Default = pd.read_excel(
    r"D:\DEHLI Machine Learning Projects\AOBD_Historical_Data\AirlineSitCapacity_Default.xlsx"
)
Aircraft_Default = Aircraft_Default.rename(columns={"ACT3": "ACT3_x"})

In [60]:
# Custom function to handle mixed data types
def convert_to_string(value):
    '''
    The goal of the function is to convert the Aircraft type to an interger.
    '''
    if isinstance(value, (int, float)):
        return str(value)
    else:
        return value

In [61]:
Aircraft_Default["ACT3_x"]=Aircraft_Default["ACT3_x"].apply(convert_to_string)

# 3.0 <span style='color:blue'> Inner Joing the two Files together</span>

In [62]:
FSDS=pd.merge(UFISAll,Kalender,on="Date",how='inner')

In [63]:
#Merge Dataframes FSDS and Aircraft_INFO
FDB_1=pd.merge(FSDS,Aircraft_INFO,on='ALACT',how='left')

In [64]:
FDB_B=FDB_1[FDB_1["ACT3_y"].isnull() & FDB_1["Seats"].isnull() & FDB_1["Airline"].isnull()]
Second_DB=FDB_B.drop(columns=["Seats","ACT3_y","Airline"])
Second_DB["ACT3_x"]=Second_DB["ACT3_x"].str.strip()

In [65]:
FDB=pd.merge(Second_DB,Aircraft_Default,on="ACT3_x",how="left")

In [66]:
#Merge Dataframes FSDS and Aircraft_INFO
FDB_New=pd.merge(FSDS,Aircraft_INFO,on='ALACT',how='inner')

In [67]:
Flight_DB1=FDB_New[['Date','Day of week', 'Weekend (0/1)', 'Holiday (0/1)',
       'Festival', 'Overlap with Weekend', 'Extended weekend','1day_after',
       '2day_after', '3day_after', '1day_before', '2day_before', '3day_before',
       '4day_before', '4day_after','ADID','STYP','PAXT','ORG3','DES3','STOA_UCT','STOD_UCT','ONBL_UCT','OFBL_UCT','Seats','FLTI',"FLNO",'STEV',"ALC2"]]

In [68]:
Flight_DB2=FDB[['Date','Day of week', 'Weekend (0/1)', 'Holiday (0/1)',
       'Festival', 'Overlap with Weekend', 'Extended weekend','1day_after',
       '2day_after', '3day_after', '1day_before', '2day_before', '3day_before',
       '4day_before', '4day_after','ADID','STYP','PAXT','ORG3','DES3','STOA_UCT','STOD_UCT','ONBL_UCT','OFBL_UCT','Seats','FLTI','FLNO','STEV',"ALC2"]]

## 3.1 Concatenating the two DFs

In [69]:
Flight_DBINFO=pd.concat([Flight_DB1,Flight_DB2], ignore_index=True)

In [70]:
Flight_DBINFO_Arrival = Flight_DBINFO[Flight_DBINFO["ADID"] == 'A'][["Date", "Day of week", "Weekend (0/1)", "Holiday (0/1)", 'Festival', 'Overlap with Weekend', 'Extended weekend','1day_after',
       '2day_after', '3day_after', '1day_before', '2day_before', '3day_before',
       '4day_before', '4day_after', "PAXT", "ORG3", "DES3", "STOA_UCT", "Seats", "FLTI",'FLNO','STEV',"ALC2"]]

In [71]:
Flight_DBINFO_Departure = Flight_DBINFO[Flight_DBINFO["ADID"] == 'D'][["Date", "Day of week", "Weekend (0/1)", "Holiday (0/1)", 'Festival', 'Overlap with Weekend', 'Extended weekend', '1day_after',
       '2day_after', '3day_after', '1day_before', '2day_before', '3day_before',
       '4day_before', '4day_after',"PAXT", "ORG3", "DES3", "STOD_UCT", "Seats", "FLTI",'FLNO','STEV',"ALC2"]]


In [72]:
print('shape of Arrival is :',len(Flight_DBINFO_Arrival))
print('shape of Depature is :',len(Flight_DBINFO_Departure))

shape of Arrival is : 24279
shape of Depature is : 21945


In [73]:
NF_Depature_Calender=NF_Depature_Calender[['Date', 'Day of week', 'Weekend (0/1)', 'Holiday (0/1)', 'Festival',
       'Overlap with Weekend', 'Extended weekend','1day_after',
       '2day_after', '3day_after', '1day_before', '2day_before', '3day_before',
       '4day_before', '4day_after',"all_boarded_pax","origin_airport_iata","destination_airport_iata","sobt_uct","all_capacity_pax","traffic_type",'flight_number','public_terminal',"airline_iata"]]

In [74]:
NF_Arrival_Calender=NF_Arrival_Calender[['Date', 'Day of week', 'Weekend (0/1)', 'Holiday (0/1)', 'Festival',
       'Overlap with Weekend', 'Extended weekend','1day_after',
       '2day_after', '3day_after', '1day_before', '2day_before', '3day_before',
       '4day_before', '4day_after',"all_boarded_pax","origin_airport_iata","destination_airport_iata","sibt_uct","all_capacity_pax","traffic_type",'flight_number','public_terminal',"airline_iata"]]

### 3.2 Renaming the columns for all the 4 DFs

In [75]:
Flight_DBINFO_Arrival=Flight_DBINFO_Arrival.rename(columns={"PAXT":"boarded pax",'ORG3':"Original Airport",'DES3':"Destination airport","STOA_UCT":"Arrival Datetime",'Seats':"Seat Capacity",'FLTI':"traffic_type","FLNO":"Flight Number","STEV":"Terminal","ALC2":"Airline Code"})

In [76]:
def remove_strips(df, column_name):
    df[column_name] = df[column_name].str.replace(" ", "").str.strip()
    return df

In [77]:
Flight_DBINFO_Arrival = remove_strips(Flight_DBINFO_Arrival, "Flight Number")

- Renaming the NF_Arrival_Calender columns

In [78]:
NF_Arrival_Calender = NF_Arrival_Calender.rename(
    columns={
        "all_boarded_pax": "boarded pax",
        'origin_airport_iata': "Original Airport",
        'destination_airport_iata': "Destination airport",
        "sibt_uct": "Arrival Datetime",
        'all_capacity_pax': "Seat Capacity",
        'leg_type': "traffic_type",
        'flight_number': "Flight Number",
        'public_terminal': "Terminal",
        "airline_iata": "Airline Code"
    })

In [79]:
Flight_DBINFO_Arrival = convert_column_int_fillna(Flight_DBINFO_Arrival,
                                                 'Seat Capacity')
Flight_DBINFO_Arrival = convert_column_int_fillna(Flight_DBINFO_Arrival,
                                                 'boarded pax')

#### 3.2.1 Concatenating the two Arrival DFS

In [80]:
Arrival_DF = pd.concat([Flight_DBINFO_Arrival, NF_Arrival_Calender],
                       ignore_index=True)

In [81]:
Flight_DBINFO_Arrival.columns

Index(['Date', 'Day of week', 'Weekend (0/1)', 'Holiday (0/1)', 'Festival',
       'Overlap with Weekend', 'Extended weekend', '1day_after', '2day_after',
       '3day_after', '1day_before', '2day_before', '3day_before',
       '4day_before', '4day_after', 'boarded pax', 'Original Airport',
       'Destination airport', 'Arrival Datetime', 'Seat Capacity',
       'traffic_type', 'Flight Number', 'Terminal', 'Airline Code'],
      dtype='object')

In [82]:
NF_Arrival_Calender.columns

Index(['Date', 'Day of week', 'Weekend (0/1)', 'Holiday (0/1)', 'Festival',
       'Overlap with Weekend', 'Extended weekend', '1day_after', '2day_after',
       '3day_after', '1day_before', '2day_before', '3day_before',
       '4day_before', '4day_after', 'boarded pax', 'Original Airport',
       'Destination airport', 'Arrival Datetime', 'Seat Capacity',
       'traffic_type', 'Flight Number', 'Terminal', 'Airline Code'],
      dtype='object')

In [83]:
Arrival_DF.shape

(363575, 24)

In [84]:
Flight_DBINFO_Departure = Flight_DBINFO_Departure.rename(
    columns={
        "PAXT": "boarded pax",
        'ORG3': "Original Airport",
        'DES3': "Destination airport",
        "STOD_UCT": "Depature Datetime",
        'Seats': "Seat Capacity",
        'FLTI': "traffic_type",
        "FLNO": "Flight Number",
        "STEV": "Terminal",
        "ALC2": "Airline Code"
    })

In [85]:
Flight_DBINFO_Departure = remove_strips(Flight_DBINFO_Departure,
                                        "Flight Number")

In [86]:
NF_Depature_Calender = NF_Depature_Calender.rename(
    columns={
        "all_boarded_pax": "boarded pax",
        'origin_airport_iata': "Original Airport",
        'destination_airport_iata': "Destination airport",
        "sobt_uct": "Depature Datetime",
        'all_capacity_pax': "Seat Capacity",
        'leg_type': "traffic_type",
        'flight_number': "Flight Number",
        "public_terminal": "Terminal",
        "airline_iata": "Airline Code"
    })

In [87]:
Flight_DBINFO_Departure = convert_column_int_fillna(Flight_DBINFO_Departure,
                                                 'Seat Capacity')
Flight_DBINFO_Departure = convert_column_int_fillna(Flight_DBINFO_Departure,
                                                 'boarded pax')

#### 3.2.1 Concatenating the two Depature DFS

In [88]:
Depature_DF = pd.concat([Flight_DBINFO_Departure, NF_Depature_Calender],
                        ignore_index=True)

In [89]:
%store Arrival_DF

Stored 'Arrival_DF' (DataFrame)


In [90]:
%store Depature_DF

Stored 'Depature_DF' (DataFrame)


In [91]:
# Record the end time
end_time = time.time()

In [92]:
total_time = end_time - start_time
print(f"Total execution time is : {total_time:.2f} seconds")
print(f"Total excecition time is : {total_time/60:.2f} minutes")

Total execution time is : 1577.27 seconds
Total excecition time is : 26.29 minutes
